### Because the output data is huge, we should take the vertical/zonal average in a separate step and save results for further analysis and plotting.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cubedsphere as cs

## Vertical mean

In [4]:
# warning: this cell takes very long to run!
# the highest-res case (C384L160) might sometimes fail

Vres_list = [20,40,80,160]#[3:]
Hres_list = [48,96,192,384]#[3:]

outputdir = "~/FV3/output/processed/vertical_mean/"

for i,Hres in enumerate(Hres_list):
    for j,Vres in enumerate(Vres_list):
        
        print('loading',Hres,Vres,end=' ')
        datadir = ("../../output/C{0}/C{0}L{1}/".format(Hres,Vres))
        ds = cs.open_FV3data(datadir,"tracer_daily")
        
        # only load the last day
        dr = ds['plume01'].isel(time=-1).copy(); dr.load()
        
        print('calculating...',end=' ')
        dr_vmean = dr.mean(dim='pfull')
        ds_vmean = dr_vmean.to_dataset()
        
        # recover lost coordinates
        ds_vmean['lat_b'] = ds['lat_b']
        ds_vmean.set_coords(["lat_b"], inplace=True)
        ds_vmean['lon_b'] = ds['lon_b']
        ds_vmean.set_coords(["lon_b"], inplace=True)
        
        print('writting...',end='\n')
        ds_vmean.to_netcdf(outputdir+'plume01_vmean_C{0}L{1}.nc'.format(Hres,Vres))

loading 384 160 calculating... writting...


## Meridional mean

In [3]:
# warning: this cell takes very long to run!

Vres_list = [20,40,80,160]#[3:]
Hres_list = [48,96,192,384]#[3:]

outputdir = "~/FV3/output/processed/meri_mean/"

for i,Hres in enumerate(Hres_list):
    for j,Vres in enumerate(Vres_list):
        
        print('loading',Hres,Vres,end=' ')
        datadir = ("../../output/C{0}/C{0}L{1}/".format(Hres,Vres))
        ds = cs.open_FV3data(datadir,"tracer_daily")
        
        # only load the last day
        dr = ds['plume01'].isel(time=-1).copy(); dr.load()
        
        print('calculating...',end=' ')
        # use finer bin at higher Hres
        dr_meri = cs.meridional_mean(dr,dlon=10.0/Hres*48)
        
        print('writting...',end='\n')
        dr_meri.to_netcdf(outputdir+'plume01_meri_C{0}L{1}.nc'.format(Hres,Vres))

loading 384 160 calculating... writting...
